In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import *
import numpy as np
import xmljson 
from xmljson import badgerfish as bf
from json import *
import xml.etree.ElementTree as ET
from io import StringIO
import webbrowser
import requests
import os as os

In [ ]:
if not os.path.exists("datas"):
    os.makedirs("datas")
directory = 'datas'

In [ ]:
def parsing_datas(base_url) :
    with urllib.request.urlopen(base_url) as url:
        s = url.read()
    root = ET.fromstring(s)

    dict_ = {}
    base = "{http://www.w3.org/2005/Atom}"
    for child in root.iter(base+'entry'):
        for children in child.iter(base+'content') :
            for properties in children :
                for subject in properties : 
                        #print(subject.text)
                    s = subject.tag.split('}')
                    if s[1] in dict_ :
                        dict_[s[1]].append(subject.text)
                    else : 
                        dict_[s[1]] = [subject.text]
    data = pd.DataFrame(dict_)
    return data

## saving_datas definition function 
- return ids of of session/vote .... 

In [ ]:
def save_datas(url, paths, paths_id,directory,final_index,change_url = False) :
        if len(paths_id[paths[0]]) == 0 :
            
            data = parsing_datas(url)
            if not data.empty :

                directory += '/'+ paths[0]
                if not os.path.exists(directory):
                    os.makedirs(directory)
                id_  = data['ID'].unique()[1:].tolist()
                final_index = final_index+id_
                data.to_csv(directory+'/'+paths[-1]+".csv")
                
        else :
            
            for index in paths_id[paths[0]] :
                new_directory = directory
                new_paths = paths_id.copy()
                new_paths.pop(paths[0],None)
                new_directory += '/'+paths[0]+'_id_'+str(index)
                if not os.path.exists(new_directory):
                    os.makedirs(new_directory)
                new_url = url
                new_change_url = change_url
                if change_url and len(paths)==2 :
                    new_url = new_url.replace('{}',index)  
                    new_change_url = False
                    print(new_url)
                final_index = save_data(new_url,paths[1:],new_paths,new_directory,final_index,new_change_url)
        return final_index    

In [ ]:
def save_data(id_,directory,id_name,parent_id= None,subject=None, url = None) :
    if url == None :
        with open('base_url.txt', 'r') as myfile:
            url=myfile.read()
    if subject != None :
        url = url.replace('[]',subject)
    if parent_id != None :   
        url = url.replace('()',parent_id)
    url = url.replace('{0}',str(np.maximum(min(id_)-1,0)))
    url = url.replace('{1}',str(max(id_)+1))
    print(url)
    data = parsing_datas(url)
    if not data.empty :
        if not os.path.exists(directory):
            os.makedirs(directory)
        index = list(map(int, data['ID'].unique().tolist()))
        data.to_csv(directory+'/'+id_name+ 'id_'+str(min(id_))+'-'+str(max(id_))+'.csv')
        return index ,data
    else :
        return None

### saving legislative datas

In [ ]:
legislative_url ="https://ws.parlament.ch/odata.svc/LegislativePeriod?$filter=LegislativePeriodNumber%20gt%20{0}%20and%20LegislativePeriodNumber%20lt%20{1}"
base_legi_directory = "datas/legi"
legi_periode_id, _  = save_data([0,100], base_legi_directory,'legi',None,None,legislative_url)
print(legi_periode_id)

### saving vote datas 

In [ ]:
base_vote_directory= "datas/Vote"
vote_id , _ = save_data(legi_periode_id,base_vote_directory,'legi','IdLegislativePeriod','Vote')
print(str(min(vote_id))+' '+str(max(vote_id)))

In [ ]:
base_session_directory= "datas/Session"
session_id, _ = save_data(legi_periode_id,base_session_directory,'Legi','LegislativePeriodNumber','Session')
print(str(min(session_id))+' '+str(max(session_id)))

In [ ]:
base_transcript_directory= "datas/Meeting"
meeting_id,_ = save_data(session_id,base_transcript_directory,'Session','IdSession','Meeting')
print(str(min(meeting_id))+' '+str(max(meeting_id)))

In [ ]:
base_subject_directory= "datas/Subject"
Subject_id, _ = save_data(meeting_id,base_subject_directory,'Meeting','IdMeeting','Subject')
print(str(min(Subject_id))+' '+str(max(Subject_id)))

In [ ]:
base_transcript_directory= "datas/Transcript"
max_Transcript_id = 206649
Transcript_id = [0]
while max(Transcript_id) < max_Transcript_id :
    Transcript_id, Transcript = save_data(Subject_id,base_transcript_directory,'Subject','IdSubject','Transcript')
    max_id = max(list(map(int,Transcript['IdSubject'])))
    Subject_id = [i for i in Subject_id if i > max_id]
    print(str(min(Transcript_id))+' '+str(max(Transcript_id)))